ETL de los negocios de google

Importamos librerias

In [1]:
import pandas as pd
import os

Nos conectamos a la base de datos

In [2]:
#leemos los archivos y los procesamos uno a uno los json que contiene cada archivo
jsons=os.listdir(r"D:\Datasets_proyecto\metadata-sitios")
for a in jsons:
    ruta=f'D:\Datasets_proyecto\metadata-sitios\\{a}'
    dfbusinessGoogle=pd.read_json(ruta,lines=True)

Filtramos los datos de ulta beauty , ya que es el alcance del proyecto

In [3]:
dfbusinessGoogle = dfbusinessGoogle[dfbusinessGoogle['name'] == 'Ulta Beauty']

Borramos la columna name , porque todos los locales se llaman igual

In [4]:
dfbusinessGoogle = dfbusinessGoogle.drop('name', axis=1)

filtramos a los negocios que estan permanentemente cerrados y borramos la columna

In [5]:
dfbusinessGoogle = dfbusinessGoogle[dfbusinessGoogle['state'] != 'Permanently closed']
dfbusinessGoogle = dfbusinessGoogle.drop('state', axis=1)

De la columna address sacamos el estado , la ciudad y el codigo postal al que pertenecen y finalmente borramos la columna

In [6]:
dfbusinessGoogle['address'] = dfbusinessGoogle['address'].str.replace('Ulta Beauty,', '').str.strip()
dfbusinessGoogle['city'] = dfbusinessGoogle['address'].str.split(',').str[1]
dfbusinessGoogle['state_PostalCode'] = dfbusinessGoogle['address'].str.split(',').str[2]
dfbusinessGoogle['state']=dfbusinessGoogle['state_PostalCode'].str.split(' ').str[1]
dfbusinessGoogle['postal_code']=dfbusinessGoogle['state_PostalCode'].str.split(' ').str[2]

In [7]:
dfbusinessGoogle = dfbusinessGoogle.drop('address', axis=1)
dfbusinessGoogle = dfbusinessGoogle.drop('state_PostalCode', axis=1)

Borramos la columna de description

In [8]:
dfbusinessGoogle = dfbusinessGoogle.drop('description', axis=1)

Borramos la columna de category

In [9]:
dfbusinessGoogle = dfbusinessGoogle.drop('category', axis=1)

Borramo la columna price ya que contiene todos sus datos nulos

In [10]:
dfbusinessGoogle = dfbusinessGoogle.drop('price', axis=1)

Borramos la columna relative_results y la de url

In [11]:
dfbusinessGoogle = dfbusinessGoogle.drop('relative_results', axis=1)
dfbusinessGoogle = dfbusinessGoogle.drop('url', axis=1)

Borramos la columna de MISC y hours

In [12]:
dfbusinessGoogle = dfbusinessGoogle.drop('MISC', axis=1)
dfbusinessGoogle = dfbusinessGoogle.drop('hours', axis=1)

Valores nulos y duplicados

In [13]:
dfbusinessGoogle.isnull().sum()

gmap_id           0
latitude          0
longitude         0
avg_rating        0
num_of_reviews    0
city              0
state             0
postal_code       0
dtype: int64

No hay valores nulos

In [14]:
# Obtener la cantidad de filas antes de eliminar duplicados
num_filas_antes = dfbusinessGoogle.shape[0]

# Eliminar duplicados basados en la columna 'address'
dfbusinessGoogle = dfbusinessGoogle.drop_duplicates()

# Obtener la cantidad de filas después de eliminar duplicados
num_filas_despues = dfbusinessGoogle.shape[0]

# Mostrar la cantidad de duplicados antes y después
print(f"Cantidad de duplicados borrados: {num_filas_antes - num_filas_despues}")


Cantidad de duplicados borrados: 0


Normalización de los nombres de las columnas

Las columnas se van a nombrar igual que las del dataset de yelp

In [15]:
dfbusinessGoogle = dfbusinessGoogle.rename(columns={'gmap_id': 'business_id', 'avg_rating': 'stars', 'num_of_reviews': 'review_count'})

Los nombres de las ciudades se va a pasar a minuscula y los espacios seran sustituidos por guion bajo

In [16]:
dfbusinessGoogle['city'] = dfbusinessGoogle['city'].str.strip().str.lower().str.replace(' ', '_')

Cambiamos las siglas por los nombres de los estados 

In [17]:
estado_mapping = {'TX': 'texas', 'MT': 'montana', 'CA': 'california', 'FL': 'florida',
                  'IL': 'illinois', 'MD': 'maryland', 'MA': 'massachusetts', 'PA': 'pennsylvania',
                  'NY': 'new_york', 'OH': 'ohio', 'OK': 'oklahoma', 'MO': 'missouri',
                  'CT': 'connecticut', 'NJ': 'new_jersey', 'NC': 'north_carolina'}

# Reemplazar las siglas por los nombres en la columna 'state_postalCode'
dfbusinessGoogle['state'] = dfbusinessGoogle['state'].replace(estado_mapping)

Cambiamos el orden de las columnas para mantener la misma estructura que la de yelp

In [22]:
nuevo_orden_columnas = ['business_id', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count']

# Reorganizar las columnas en el DataFrame
dfbusinessGoogle = dfbusinessGoogle[nuevo_orden_columnas]

Agregamos la columna source para identificar que estos datos provienen del dataset de google

In [24]:
dfbusinessGoogle['source']="G"

In [25]:
dfbusinessGoogle.head()

,business_id,city,state,postal_code,latitude,longitude,stars,review_count,source
3858,0x864c394f020aaa49:0x20e59a02a019a45d,frisco,texas,75034,33.173242,-96.843504,3.9,143,G
37741,0x5342381c6ddf30e3:0xfbe922695b89d6de,great_falls,montana,59404,47.518201,-111.315605,4.1,164,G
53920,0x808fe985ad5d01df:0x20e8c372f20b006c,livermore,california,94551,37.696312,-121.847587,4.3,66,G
72836,0x88d91958f9f6f4ab:0x111e84f49744687d,boca_raton,florida,33434,26.364980,-80.202167,4.0,158,G
82511,0x880fa30ab6b747f3:0x2a88bf8178a2c164,kildeer,illinois,60047,42.162774,-88.050782,3.8,8,G
